## 1. Read Basic State Model

By reading the Excel file and generating the python state model, the state model can be stored as '.pkl' file.
The '.pkl' file is faster in reading than the Excel file, but should be updated if the Excel file is changed.
The '.pkl' file is generated automatically after the Excel file is deserialized.

In [ ]:
from pathlib import Path
import pandas as pd

from ofact.planning_services.model_generation.persistence import deserialize_state_model
from ofact.twin.repository_services.deserialization.order_types import OrderType

from projects.bicycle_world.settings import PROJECT_PATH
print(PROJECT_PATH)

In [ ]:
path_to_model = "scenarios/current/models/twin/"
state_model_file_name = "base_wo_material_supply.xlsx"

state_model_file_path = Path(str(PROJECT_PATH), path_to_model + state_model_file_name)
state_model_generation_settings = {"order_generation_from_excel": False,
                                   "customer_generation_from_excel": True,
                                   "customer_amount": 5, 
                                   "order_amount": 20,
                                   "order_type": OrderType.PRODUCT_CONFIGURATOR}
state_model = deserialize_state_model(state_model_file_path, persistence_format="xlsx",
                                      state_model_generation_settings=state_model_generation_settings)

### Get an insight in the available orders:

In [ ]:
available_orders = state_model.get_orders()
first_n_orders = 1
for order in available_orders[:first_n_orders]:
    print(f"The order with the identifier '{order.identifier}' "
          f"ordered on '{order.order_date}' is planned to deliver on '{order.delivery_date_planned}'.")

    print(f"\n The requested features of the order are ({len(order.features_requested)}): ")
    features_requested = pd.DataFrame([[feature.feature_cluster.name, feature.name] 
                                       for feature in order.features_requested],
                                      columns=["Feature Cluster", "Feature Name"])
    print(features_requested)

    print(f"\n The completed features of the order are ({len(order.features_completed)}): ")
    features_completed = pd.DataFrame([[feature.feature_cluster.name, feature.name]
                                       for feature in order.features_completed],
                                      columns=["Feature Cluster", "Feature Name"])
    print(features_completed)
    print("\n\n\n")

### Get an insight in the available resources:

In [ ]:
all_resources = state_model.get_all_resources()

print("How many resources are available in the state model?")
print(len(all_resources))

warehouses = state_model.get_warehouses()
work_stations = state_model.get_work_stations()

storages = state_model.get_storages()

active_moving_resources = state_model.get_active_moving_resources()
passive_moving_resources = state_model.get_passive_moving_resources()


### Get an insight in the available parts:

In [ ]:
parts = state_model.get_parts()

print("How many parts are available in the state model?")
print(len(parts))

print("How many different parts are available in the state model?")
print(len(set(part.entity_type for part in parts)))

### Get an insight in the available processes:

In [ ]:
all_processes = state_model.get_all_processes()

print("How many processes are available in the state model?")
print(len(all_processes))

n = 1
value_added_processes = state_model.get_value_added_processes()
print("Which value added processes are in the state model?")

value_added_processes_df = pd.DataFrame([value_added_process.name
                                         for value_added_process in value_added_processes[:min(n, len(value_added_processes))]],
                                        columns=["Value Added Process Name"])

processes = state_model.get_processes()
print("Which normal processes are in the state model?")

processes_df = pd.DataFrame([process.name
                             for process in processes[:min(n, len(processes))]],
                             columns=["Process Name"])
process_df = pd.concat([value_added_processes_df, processes_df],
                       axis=1)
print(process_df)

## 2. Data from Shop Floor

Take a look in the available data files from our "bicycle world" scenario.
- Event Log/ Execution Log Data
- Order Data

In [ ]:
import pandas as pd

In [ ]:
# read the source files
execution_log_df = pd.read_excel("../data/input/executions.xlsx")

print(execution_log_df.columns.values)
print(execution_log_df.head())

In the execution log file, we have a list of all executed processes.
Each row has the following data:
- process information (link to the executed process identification and/ or name)
- time information (start and end time of the process)
- order information (link to the associated order)
- resource information (link to the associated resource(s))
- (input) part information (link to the associated part(s))
- transition information (origin and destination resource required to specify the transport or transfer)
- quality information (specify the resulting quality of the transformed parts)

These data entries are required to update the state model of the digital twin.
However, this is just one example of how the data entry might look like.
Diverging data structures are also possible, such as standard event logs.
In event logs, a process can have multiple entries, e.g., an entry is created for each event.
Data gaps (missing data) can also be handled as part of the data integration.


In [ ]:
# read the source files
order_pool_df = pd.read_excel("../data/input/orders.xlsx")

print(order_pool_df.columns.values)
print(order_pool_df.head())

In the order pool log file, we have a list of all orders.
They could be already finished, currently in progress or planned.
Each row has the following data:
- order information (link to the order identification and/ or name)
- customer information (link to the associated customer)
- price information (How many does the order cost?)
- product information (link to the associated product (part) and/or the product class (type of the product))
- time information (timestamps of the order lifecycle 
    - ('Order Date', 'Release Date', 'Delivery Date Requested', 'Delivery Date Planned', 'Delivery Date Actual'))
- urgency information (Is the order urgent?)
- feature information (describes the product specifications chosen by the customer)
    - Features are mapped to processes (in the static state model). 
      The order is finished through executing a set of processes that are required to add the chosen features to the product

These data entries are required to update the state model of the digital twin.
However, this is just one example of how the data entry might look like.
Diverging data structures are also possible.
For example, if the order in a use case is based on BOMs or processes rather than on features, features could be added artificially.
As mentioned previously for the event logs, data gaps (missing data) can be handled as part of the data integration.


## 3. Update the digital twin state model based on the data source model.

In [ ]:
from datetime import datetime
from pathlib import Path

from ofact.twin.state_model.model import _transform_order_pool_list_to_np_array
from ofact.settings import ROOT_PATH

from projects.bicycle_world.scenarios.current.data_integration.update_digital_twin import get_digital_twin_updated

In [ ]:
state_model.orders = _transform_order_pool_list_to_np_array([])  # shouldn't be mandatory

start_datetime = datetime(2024, 10, 22, 8)
end_datetime = None  # datetime(2024, 10, 29)

data_source_model_path = Path(str(PROJECT_PATH),
                              "scenarios/current/models/data_source/data_source_model.xlsx")

updated_state_model = get_digital_twin_updated(ROOT_PATH, PROJECT_PATH,
                                               state_model, start_datetime, end_datetime, data_source_model_path)

## 4. Persist the updated state model

In addition to the static data modeled in the Excel sheets, we have added dynamic data to the state model.
For the serialization, we need to add the "dynamics" flag.

In [ ]:
from ofact.twin.repository_services.persistence import serialize_state_model

target_file_path = "../../tutorial/updated_state_model.pkl"
serialize_state_model(state_model=updated_state_model, target_file_path=target_file_path,
                      dynamics=True)